In [15]:
import numpy as np

In [16]:
# матрица, близкая к диагональной
A1 = np.array([
    [10, 1, 0.5],
    [0.5, 8, 1],
    [0.5, 0.5, 6]
], dtype=float)
b1 = np.array([11.5, 9.5, 7.0], dtype=float)

# матрица, близкая к нижней треугольной
A2 = np.array([
    [7, 0, 0],
    [3, 6, 0],
    [2, 1, 5]
], dtype=float)
b2 = np.array([7, 9, 8], dtype=float)

# симметричная положительно определенная матрица
A3 = np.array([
    [4, 1, 0.5],
    [1, 5, 1],
    [0.5, 1, 6]
], dtype=float)
b3 = np.array([5.5, 7, 7.5], dtype=float)

# матрица для сравнения сходимости (2x2)
A4 = np.array([
    [2, 1.5],
    [1.5, 2]
], dtype=float)
b4 = np.array([3.5, 3.5], dtype=float)

test_cases = [
    (A1, b1, "Диагонально доминирующая матрица"),
    (A2, b2, "Близкая к нижней треугольной"),
    (A3, b3, "Симметричная положительно определенная"),
    (A4, b4, "Матрица 2x2 для сравнения")
]

In [17]:
# метод зейдля
def seidel_method(A, b, x0, epsilon=1e-6, max_iter=1000, verbose=True):
    n = len(b)
    x = np.array(x0, dtype=float)
    B = np.zeros((n, n))
    c = np.zeros(n)
    for i in range(n):
        c[i] = b[i] / A[i, i]
        for j in range(n):
            if i != j:
                B[i, j] = -A[i, j] / A[i, i]
    if verbose:
        print("=" * 50)
        print("МЕТОД ЗЕЙДЕЛЯ")
        print("=" * 50)
        print(f"Начальное приближение: {x0}")
    iterations = 0
    for k in range(max_iter):
        x_new = x.copy()
        for i in range(n):
            sum1 = sum(B[i, j] * x_new[j] for j in range(i))
            sum2 = sum(B[i, j] * x[j] for j in range(i+1, n))
            x_new[i] = c[i] + sum1 + sum2
        diff = np.linalg.norm(x_new - x, np.inf)
        iterations += 1
        if verbose:
            print(f"Итерация {k+1}:")
            print(f"  x = [{', '.join(f'{val:.6f}' for val in x_new)}]")
            print(f"  ‖x^(k+1) - x^(k)‖∞ = {diff:.6e}")
        if diff < epsilon:
            if verbose:
                print(f"✓ Достигнута точность {epsilon} за {k+1} итераций")
            return x_new, k+1
        x = x_new
    if verbose:
        print(f"Достигнут максимум итераций ({max_iter})")
    return x, max_iter

In [18]:
# метод простой интерации
def simple_iteration_method(A, b, x0, epsilon=1e-6, max_iter=1000, verbose=True):
    n = len(b)
    x = np.array(x0, dtype=float)
    B = np.zeros((n, n))
    c = np.zeros(n)
    for i in range(n):
        c[i] = b[i] / A[i, i]
        for j in range(n):
            if i != j:
                B[i, j] = -A[i, j] / A[i, i]
    if verbose:
        print("=" * 50)
        print("МЕТОД ПРОСТОЙ ИТЕРАЦИИ")
        print("=" * 50)
        print(f"Начальное приближение: {x0}")
    iterations = 0
    for k in range(max_iter):
        x_new = B @ x + c
        diff = np.linalg.norm(x_new - x, np.inf)
        iterations += 1
        if verbose and k < 3:
            print(f"Итерация {k+1}:")
            print(f"  x = [{', '.join(f'{val:.6f}' for val in x_new)}]")
            print(f"  ‖x^(k+1) - x^(k)‖∞ = {diff:.6e}")
        if diff < epsilon:
            if verbose and k >= 3:
                print(f"Достигнута точность {epsilon} за {k+1} итераций")
            return x_new, k+1
        x = x_new
    if verbose:
        print(f"Достигнут максимум итераций ({max_iter})")
    return x, max_iter

In [19]:
def solve_with_precisions(A, b, description):
    print("\n" + "=" * 80)
    print(f"СИСТЕМА: {description}")
    print("=" * 80)
    print(f"A = \n{A}")
    print(f"b = {b}")
    print(f"Размер: {A.shape[0]}x{A.shape[1]}")
    x0 = [0] * len(b)
    results = {}
    precisions = [1e-3, 1e-6]
    for eps in precisions:
        print(f"\n{'='*40}")
        print(f"ТОЧНОСТЬ: {eps}")
        print('='*40)
        print("\n[Метод Зейделя]")
        x_seidel, iter_seidel = seidel_method(A, b, x0, epsilon=eps, max_iter=1000, verbose=True)
        print("\n[Метод простой итерации]")
        x_simple, iter_simple = simple_iteration_method(A, b, x0, epsilon=eps, max_iter=1000, verbose=True)
        diff_solutions = np.linalg.norm(x_seidel - x_simple, np.inf)
        print(f"\nРазница решений: {diff_solutions:.2e}")
        results[eps] = {
            'seidel': {'iterations': iter_seidel, 'solution': x_seidel},
            'simple': {'iterations': iter_simple, 'solution': x_simple},
            'diff': diff_solutions
        }
    return results

In [20]:
def compare_methods(all_results):
    print("\n" + "=" * 80)
    print("ИТОГОВОЕ СРАВНЕНИЕ МЕТОДОВ")
    print("=" * 80)
    print("\nСРАВНЕНИЕ ПО КОЛИЧЕСТВУ ИТЕРАЦИЙ:")
    print("-" * 70)
    print(f"{'Система':<35} {'ε=1e-3':<20} {'ε=1e-6':<20}")
    print(f"{'':<35} {'Зейд.':<10} {'Прост.':<10} {'Зейд.':<10} {'Прост.':<10}")
    print("-" * 70)
    for description, results in all_results.items():
        eps1e3 = results[1e-3]
        eps1e6 = results[1e-6]
        print(f"{description:<35} "
              f"{eps1e3['seidel']['iterations']:<10} "
              f"{eps1e3['simple']['iterations']:<10} "
              f"{eps1e6['seidel']['iterations']:<10} "
              f"{eps1e6['simple']['iterations']:<10}")
    print("\n" + "=" * 80)
    print("АНАЛИЗ РЕЗУЛЬТАТОВ:")
    print("=" * 80)
    total_speedup_1e3 = 0
    total_speedup_1e6 = 0
    count = 0
    for description, results in all_results.items():
        eps1e3 = results[1e-3]
        eps1e6 = results[1e-6]
        iter_seidel_1e3 = eps1e3['seidel']['iterations']
        iter_simple_1e3 = eps1e3['simple']['iterations']
        iter_seidel_1e6 = eps1e6['seidel']['iterations']
        iter_simple_1e6 = eps1e6['simple']['iterations']
        speedup_1e3 = 0
        speedup_1e6 = 0
        if iter_simple_1e3 > 0:
            speedup_1e3 = ((iter_simple_1e3 - iter_seidel_1e3) / iter_simple_1e3) * 100
            total_speedup_1e3 += speedup_1e3
        if iter_simple_1e6 > 0:
            speedup_1e6 = ((iter_simple_1e6 - iter_seidel_1e6) / iter_simple_1e6) * 100
            total_speedup_1e6 += speedup_1e6
        count += 1
        print(f"\n{description}:")
        if iter_seidel_1e3 < iter_simple_1e3:
            print(f"  При ε=1e-3: Метод Зейделя быстрее на {iter_simple_1e3 - iter_seidel_1e3} итераций "
                  f"({speedup_1e3:.1f}% быстрее)")
        elif iter_seidel_1e3 > iter_simple_1e3:
            print(f"  При ε=1e-3: Метод простой итерации быстрее на {iter_seidel_1e3 - iter_simple_1e3} итераций")
        else:
            print(f"  При ε=1e-3: Методы одинаковы по скорости")

        if iter_seidel_1e6 < iter_simple_1e6:
            print(f"  При ε=1e-6: Метод Зейделя быстрее на {iter_simple_1e6 - iter_seidel_1e6} итераций "
                  f"({speedup_1e6:.1f}% быстрее)")
        elif iter_seidel_1e6 > iter_simple_1e6:
            print(f"  При ε=1e-6: Метод простой итерации быстрее на {iter_seidel_1e6 - iter_simple_1e6} итераций")
        else:
            print(f"  При ε=1e-6: Методы одинаковы по скорости")
    if count > 0:
        avg_speedup_1e3 = total_speedup_1e3 / count
        avg_speedup_1e6 = total_speedup_1e6 / count

In [21]:
def main():
    all_results = {}
    for A, b, description in test_cases:
        results = solve_with_precisions(A, b, description)
        all_results[description] = results
    compare_methods(all_results)
if __name__ == "__main__":
    main()


СИСТЕМА: Диагонально доминирующая матрица
A = 
[[10.   1.   0.5]
 [ 0.5  8.   1. ]
 [ 0.5  0.5  6. ]]
b = [11.5  9.5  7. ]
Размер: 3x3

ТОЧНОСТЬ: 0.001

[Метод Зейделя]
МЕТОД ЗЕЙДЕЛЯ
Начальное приближение: [0, 0, 0]
Итерация 1:
  x = [1.150000, 1.115625, 0.977865]
  ‖x^(k+1) - x^(k)‖∞ = 1.150000e+00
Итерация 2:
  x = [0.989544, 1.003420, 1.000586]
  ‖x^(k+1) - x^(k)‖∞ = 1.604557e-01
Итерация 3:
  x = [0.999629, 0.999950, 1.000035]
  ‖x^(k+1) - x^(k)‖∞ = 1.008437e-02
Итерация 4:
  x = [1.000003, 0.999995, 1.000000]
  ‖x^(k+1) - x^(k)‖∞ = 3.746064e-04
✓ Достигнута точность 0.001 за 4 итераций

[Метод простой итерации]
МЕТОД ПРОСТОЙ ИТЕРАЦИИ
Начальное приближение: [0, 0, 0]
Итерация 1:
  x = [1.150000, 1.187500, 1.166667]
  ‖x^(k+1) - x^(k)‖∞ = 1.187500e+00
Итерация 2:
  x = [0.972917, 0.969792, 0.971875]
  ‖x^(k+1) - x^(k)‖∞ = 2.177083e-01
Итерация 3:
  x = [1.004427, 1.005208, 1.004774]
  ‖x^(k+1) - x^(k)‖∞ = 3.541667e-02
Достигнута точность 0.001 за 6 итераций

Разница решений: 2.48e-